In [27]:
#the system instruction our vertex ai is working on.
{
  "system_instruction": {
    "role": "legal document assistant",
    "content": [
      "You are a highly reliable legal document assistant specialized in answering questions about legal documents.",
      "Your role is to extract, summarize, and clarify content from provided documents without altering meaning.",
      "Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.",
      "Do not provide legal advice or personal opinions. Remain neutral and factual.",
      "When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.",
      "Use plain and concise language, but preserve legal precision.",
      "If a question is ambiguous, ask for clarification rather than assuming."
    ]
  }
}


{'system_instruction': {'role': 'legal document assistant',
  'content': ['You are a highly reliable legal document assistant specialized in answering questions about legal documents.',
   'Your role is to extract, summarize, and clarify content from provided documents without altering meaning.',
   'Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.',
   'Do not provide legal advice or personal opinions. Remain neutral and factual.',
   'When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.',
   'Use plain and concise language, but preserve legal precision.',
   'If a question is ambiguous, ask for clarification rather than assuming.']}}

In [ ]:
from flask import Flask, request, jsonify
from google.cloud import storage
import os

app = Flask(__name__)

# Set your Google Cloud credentials and bucket name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service-account.json"
BUCKET_NAME = "your-bucket-name"

@app.route('/upload-pdf', methods=['POST'])
def upload_pdf():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400
    if file and file.filename.lower().endswith('.pdf'):
        client = storage.Client()
        bucket = client.bucket(BUCKET_NAME)
        blob = bucket.blob(file.filename)
        blob.upload_from_file(file)
        return jsonify({"message": "File uploaded", "gcs_uri": f"gs://{BUCKET_NAME}/{file.filename}"}), 200
    return jsonify({"error": "Invalid file type"}), 400

if __name__ == '__main__':
    app.run(debug=True)

In [28]:
#this is how we are going to fetch the files, this is how the workflow is going to be, uploaded file will be sent to our bucket.
#and the GS util URI will be returned which can be sent to the Vertex AI API for processing.

from google.cloud import storage

def upload_blob_and_get_uri(bucket_name, source_file_name, destination_blob_name, project_id):
    """Uploads a file to the specified bucket and returns its gs:// URI."""
    # Pass the project_id explicitly
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    gs_uri = f"gs://{bucket_name}/{destination_blob_name}"

    print(f"File {source_file_name} uploaded to {destination_blob_name} in bucket {bucket_name}.")
    print(f"GS URI: {gs_uri}")

    return gs_uri


In [ ]:

your_bucket_name = "legal-doc-bucket1"  # Replace with your actual bucket name
local_file_path = r"C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf" # Replace with the path to the file you want to upload
destination_object_name = "generative-ai_pdf_certificate_of_incoporation.pdf" # The name you want the file to have in the bucket
project_id = "sodium-coil-470706-f4" # Use your actual project ID

# Call the function to upload and get the URI
uploaded_uri = upload_blob_and_get_uri(
    your_bucket_name,
    local_file_path,
    destination_object_name,
    project_id
)

if uploaded_uri:
    print(f"\nSuccessfully obtained GS URI: {uploaded_uri}")


File C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf uploaded to generative-ai_pdf_certificate_of_incoporation.pdf in bucket legal-doc-bucket1.
GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf

Successfully obtained GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf


In [30]:
#the bot has been implemented below.
from google import genai
from google.genai import types
import base64

def generate():
  client = genai.Client(
      vertexai=True,
      project="sodium-coil-470706-f4",
      location="global", 
  )

  document1 = types.Part.from_uri(
      file_uri=uploaded_uri, #the uploaded uri we fetched from the above cell would be passed here.
      mime_type="application/pdf",
  )
  
  text1 = types.Part.from_text(text="""What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic.""")
  si_text1 = """\"You are a highly reliable legal document assistant specialized in answering questions about legal documents.\",
      \"Your role is to extract, summarize, and clarify content from provided documents without altering meaning.\",
      \"Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.\",
      \"Do not provide legal advice or personal opinions. Remain neutral and factual.\",
      \"When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.\",
      \"Use plain and concise language, but preserve legal precision.\",
      \"If a question is ambiguous, ask for clarification rather than assuming.\""""

  model = "gemini-2.5-flash-lite"
  contents = [
    types.Content(
      role="user",
      parts=[
        document1,
        text1
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")

generate()

The certificate of incorporation/authority will be revoked 60 days from the date of the notice, which is June 19, 2023. Therefore, the revocation date is August 18, 2023. This is stated in the third paragraph of the document: "Pursuant to the provisions set forth in R.I.G.L. 7-1.2-1310 and 7-1.2-1414, the Certificate of Incorporation/Authority of the above-named entity will be revoked after 60 days from the date of this notice for failure to file the Annual Report."